In [1]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import scipy as sci
import matplotlib.ticker as ticker
import random as rand
import matplotlib.pyplot as plt
import cmath as cm
from IPython.display import display, Latex
import MLE_functions as fn
from sympy import pprint
from scipy.optimize import minimize as GDlib

rho:
⎡0.5   0   0⎤
⎢           ⎥
⎢ 0   0.5  0⎥
⎢           ⎥
⎣ 0    0   0⎦

 rho_num 
 [[ 4.974e-01+0.00000000e+00j  2.700e-03+2.94448637e-03j
  -2.800e-03+1.93989690e-02j]
 [ 2.700e-03-2.94448637e-03j  5.034e-01+2.46508352e-18j
   2.000e-04+1.73205081e-02j]
 [-2.800e-03-1.93989690e-02j  2.000e-04-1.73205081e-02j
  -8.000e-04-4.94895698e-20j]]

 prob vec [0.0833333333, 0.0833333333, 0.1666666667, 0.0833333333, 0.0833333333, 0.1666666667, 0.0833333333, 0.0833333333, 0.1666666667]

 nj vec [837, 837, 1659, 788, 879, 1664, 888, 767, 1681]

 sum pj, sum nj  0.9999999999000001 10000


In [22]:
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')
c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2

def Creating_states( params = [0, m.pi/2,0, 0, m.pi/4], input_type = 'theta', Abstract = False):    # coeff list like [a0,a1,b0,b1]
    a0, a1 = sym.cos(theta1), sym.exp(sym.I*phi1)*sym.sin(theta1)
    b0, b1 = sym.cos(theta2), sym.exp(sym.I*phi2)*sym.sin(theta2)
    alpha = sym.symbols('alpha')        # Doesn't work without it for some reason..
    if Abstract == False:
        alpha = params[4] 
        a0, a1, b0, b1 = a0.subs({theta1 : params[0]}), a1.subs({theta1 : params[0], phi1 : params[2]}), b0.subs({theta2 : params[1]}), b1.subs({theta2 : params[1], phi2: params[3]})
        alpha = params[4] 
        coeff_raw = [a0,a1, b0, b1]
        coeff = [i if abs(i)> 1e-9 else 0 for i in coeff_raw]
        a0, a1, b0, b1 = coeff[0],coeff[1],coeff[2],coeff[3]
    psi0, psi1 = sym.Matrix([a0,a1]), sym.Matrix([b0,b1])      # defining states
    
    psi0psi0_list, psi1psi1_list = [psi0[0]*psi0[0], psi0[1]*psi0[1], (np.sqrt(2)*psi0[0]*psi0[1]) ],  [psi1[0]*psi1[0],psi1[1]*psi1[1], (np.sqrt(2)*psi1[0]*psi1[1])] # @@@ the square rooting is simplistic. see notebook for better
    # note the order is now how prof Hillery is using ie. [|00>, |11>, |+>] (instead of the previous order [|00>, |+>, |11> ])
    psi0psi0, psi1psi1 = sym.Matrix(psi0psi0_list), sym.Matrix(psi1psi1_list)
    rho = sym.cos(alpha)**2*(psi0psi0*psi0psi0.T)+ sym.sin(alpha)**2*(psi1psi1*psi1psi1.T)
    return([[psi0,psi1], [psi0psi0,psi1psi1], rho])

# # analytic formuale for the probabilities, as given in the notes
pr1 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1)*s1*c1)+w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2)*s2*c2))
pr2 = (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1)*c1*s1) + w2*c2**2*(1+s2**2*-2*m.sqrt(2)*sym.cos(phi2)*c2*s2))
pr3 = (1/6)*(w1*(c1**4+s1**4 -2*sym.cos(2*phi1)*s1**2*c1**2)+ w2*(c2**4+ s2**4 - 2*sym.cos(2*phi2)*s2**2*c2**2))
pr4 = (1/6)*(w1*s1**2*(1+c1**2 -2*m.sqrt(2)*sym.cos(phi1+2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2+2*m.pi/3)*s2*c2))
pr5= (1/6)*(w1*c1**2*(1+s1**2 -2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr6 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1 - 2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2 - 2*m.pi/3)*c2**2*s2**2))
pr7 = (1/6)*(w1*s1**2*(1+c1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*s1*c1)+ w2*s2**2*(1+c2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*s2*c2))
pr8 = (1/6)*(w1*c1**2*(1+s1**2-2*m.sqrt(2)*sym.cos(phi1-2*m.pi/3)*c1*s1)+ w2*c2**2*(1+s2**2-2*m.sqrt(2)*sym.cos(phi2-2*m.pi/3)*c2*s2))
pr9 = (1/6)*(w1*(c1**4+s1**4-2*sym.cos(2*phi1+2*m.pi/3)*c1**2*s1**2)+ w2*(c2**4+s2**4-2*sym.cos(2*phi2+2*m.pi/3)*c2**2*s2**2))
pr = [pr1, pr2,pr3, pr4,pr5, pr6,pr7, pr8,pr9]      # list of analytic prob.

w = m.e**((2/3)*m.pi*(1j))     # third root of unity
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))  # an array of POVM direction vectors
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]   # a list of POVM matrices

def num_experiment(N = 10000, params = [0,m.pi/2,0,0,m.pi/4], show_calcs = False, seed = None):
    if seed is not None:
        rand.seed(seed)
    
    creation = Creating_states( params = params, Abstract = 0) # theoretical rho
    states = creation[0]
    rho = creation[-1]
    # print(rho)
    # print(states)
    prob_vec_sympy =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)]    # created list of Th probabilities
    prob_vec_raw = [(float(i.as_real_imag()[0])+float(i.as_real_imag()[1])*1j) for i in prob_vec_sympy]  # this is to avoid error, to convert sympy float to ordinary number
    prob_vec = [round(i.real, 10) for i in prob_vec_raw if abs(i.imag) < .0001]          # cleaned up theoretical prob vector

    POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']      # symbols to indicate collapsed direction
    #prob distribution is simply the corresponding elements of the prob_vec
    collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials
    nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]
    pj_num_vec = [i/N for i in nj_vec]                                  # numerical prob vector     
    if show_calcs == True: 
        print("\n rho \n", rho)
        print("\n prob vec", prob_vec)
        print("\n nj vec",nj_vec)
        print("\n sum pj, sum nj ", sum(prob_vec), sum(nj_vec))
    return [states, rho, prob_vec, nj_vec]

#fidelity function using parameters
def fid(params1 = [0,m.pi/2,0,0,m.pi/4], params2 = [0,m.pi/2,0,0,m.pi/4]):
    # Generate the quantum states using the Creating_states function
    states1 = Creating_states(params1)[0]
    states2 = Creating_states(params2)[0]
    
    # Calculate the fidelities between the corresponding states
    fid0_sym = (abs(states1[0].H * states2[0])**2).evalf()
    fid1_sym = (abs(states1[1].H * states2[1])**2).evalf()
    
    # Extract the real part of the fidelity values
    fid0 = complex(fid0_sym[0]).real
    fid1 = complex(fid1_sym[0]).real
    
    return [fid0, fid1]

In [18]:
def lnl2(search_params , nj = [797, 887, 1657, 825, 824, 1673, 876, 837, 1624]):       # default for seed = 42
        global pr                                           # importing the Analytical probabilities
        lnL_list = [nj[i]*sym.log(pr[i]) for i in range(len(pr))]        
        lnL_abst = sum(lnL_list).evalf()                                  # created the likelihood fucntion
        lnL = lnL_abst.subs({theta1: search_params[0], theta2: search_params[1] , phi1: search_params[2] , phi2: search_params[3] , alpha: search_params[4] })
        cost = -(lnL.evalf())
        return cost


In [21]:
bounds = [(-1,1), (-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180)]
x1 = GDlib(lnl2, [0,1,0,0,m.pi/4], args =([797, 887, 1657, 825, 824, 1673, 876, 837, 1624]), bounds = bounds )
x2 = GDlib(lnl2, [.1,.1,0,1,m.pi/8], args =([797, 887, 1657, 825, 824, 1673, 876, 837, 1624]), bounds = bounds )
x2 = GDlib(lnl2, [-1,0,2,.1,0], args =([797, 887, 1657, 825, 824, 1673, 876, 837, 1624]), bounds = bounds )
x3 = GDlib(lnl2, [2,-.2,-.3,.1,m.pi/16], args =([797, 887, 1657, 825, 824, 1673, 876, 837, 1624]), bounds = bounds )
x4 = GDlib(lnl2, [2.2,-.8,-.9,3,m.pi/3], args =([797, 887, 1657, 825, 824, 1673, 876, 837, 1624]), bounds = bounds )
print(x1, '\n')
print(x2, '\n')
print(x3, '\n')
print(x4, '\n')

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 20564.3521338011
        x: [-3.781e-01  1.546e+00  1.345e+00  8.176e-01  7.364e-01]
      nit: 29
      jac: [ 4.002e-03  2.183e-02 -4.366e-03  3.638e-04 -6.548e-03]
     nfev: 198
     njev: 33
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64> 

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 21471.7547461676
        x: [-1.000e+00  3.813e-01  3.752e-02 -1.106e+00  9.940e-01]
      nit: 28
      jac: [ 1.371e+03  1.637e-02  4.729e-03  5.093e-03 -5.457e-03]
     nfev: 354
     njev: 59
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64> 

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 23184.7778589667
        x: [ 1.000e+00 -1.419e+00 -1.406e+00  3.159e+00  3.159e+00]
      nit: 17
      jac: [-5.446e+03  2.910e-03  1.746e-02 -4.220e-01 -2.300e+02]
     nfev: 108
     njev: 18

In [24]:
pp  = [[-3.781e-01,  1.546e+00,  1.345e+00,  8.176e-01,  7.364e-01],[-1.000e+00,  3.813e-01,  3.752e-02, -1.106e+00,  9.940e-01],[ 1.000e+00, -1.419e+00, -1.406e+00,  3.159e+00,  3.159e+00],[ 3.718e-01,  1.550e+00, -1.553e+00,  1.053e+00,  7.353e-01]]

for i in pp:
    print(fid(i))

[0.8637243363111068, 0.9993852681838642]
[0.2919265817264289, 0.13847881647131927]
[0.2919265817264289, 0.9771343121722359]
[0.8680181711099467, 0.9995675751366397]


In [15]:
def lnl2(search_params):       # default for seed = 42
        global pr                                           # importing the Analytical probabilities
        nj = [797, 887, 1657, 825, 824, 1673, 876, 837, 1624]
        lnL_list = [nj[i]*sym.log(pr[i]) for i in range(len(pr))]        
        lnL_abst = sum(lnL_list).evalf()                                  # created the likelihood fucntion
        lnL = lnL_abst.subs({theta1: search_params[0], theta2: search_params[1] , phi1: search_params[2] , phi2: search_params[3] , alpha: search_params[4] })
        cost =  -(lnL.evalf())
        return cost
GDlib(lnl2, [0,.1,0,1,.1], bounds = bounds)

  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 23771.3782160160
        x: [-1.000e+00  3.159e+00  4.344e-01 -1.553e+00  3.159e+00]
      nit: 5
      jac: [ 5.490e+03 -2.636e+00 -3.638e-04 -2.692e-02 -1.864e+02]
     nfev: 48
     njev: 8
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>

In [6]:
#rough
params0 = [.1, .3, 0, 0, m.pi/4]
nj_vec = [1678, 0, 1639, 1656, 0, 1662, 1723, 0, 1642]
bounds = [(-1,1), (-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180),(-m.pi/2+m.pi/180,m.pi+m.pi/180)]
print(sum(nj_vec))
def lnL2(search_params = params0):
        global nj_vec                                     # Importing the collapse counts From the outer function
        global pr                                           # importing the Analytical probabilities
        lnL_list = [nj_vec[i]*sym.log(pr[i]) for i in range(len(pr))]        
        lnL_abst = sum(lnL_list).evalf()                                  # created the likelihood fucntion
        lnL = lnL_abst.subs({theta1: search_params[0], theta2: search_params[1] , phi1: search_params[2] , phi2: search_params[3] , alpha: search_params[4] })
        return -(lnL.evalf())
opt = GDlib(lnL2, params0, bounds= bounds)
opt

10000


  message: CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
  success: True
   status: 0
      fun: 23098.9783021071
        x: [ 1.000e+00  3.159e+00 -1.215e+00 -1.553e+00  3.159e+00]
      nit: 6
      jac: [-1.771e+04  1.670e-01  2.183e-03  2.183e-03 -9.329e+01]
     nfev: 66
     njev: 11
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>

In [3]:
#Untouchable
# Gradeint search using library method
def GD_search_lib(N = 10000, params = [0,m.pi/2,0,0,m.pi/4] , params0 = [.1, .3, 0, 0, m.pi/4], show_calcs = False):
    bounds = [(-m.pi,m.pi+.1), (-m.pi,m.pi+.1) ,(-m.pi,m.pi+.1),(-m.pi,m.pi+.1),(0,m.pi/2+.1)]
    num_expt = num_experiment(N = N, params = params)
    states, rho, th_pr = num_expt[0], num_expt[1], num_expt[2] # for later
    nj_vec = num_expt[3]
    global pr                                           # importing the Analytical probabilities
    lnL_list = [nj_vec[i]*sym.log(pr[i]) for i in range(len(pr))]     
    lnL_abst = sum(lnL_list).evalf()                                  
    def lnL(sp = params0):                                    #s_p stands for search parameters
        lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1] , phi1: sp[2] , phi2: sp[3] , alpha: sp[4] })
        neg_likelihood = -(lnL.evalf())
        return neg_likelihood 
    #grad function
    def grad(p=[0,m.pi/2,0,0,m.pi/4] , s=(1/500), show_calcs = False):    # "point" is the list of parameters
        del_theta1 = lnL(sp = [p[0]+s,p[1],p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_theta2 = lnL(sp = [p[0],p[1]+s,p[2],p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi1   = lnL(sp = [p[0],p[1],p[2]+s,p[3],p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_phi2   = lnL(sp = [p[0],p[1],p[2],p[3]+s,p[4]]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])
        del_alpha  = lnL(sp = [p[0],p[1],p[2],p[3],p[4]+s]) - lnL(sp = [p[0],p[1],p[2],p[3],p[4]])

        grad = [del_theta1/s,del_theta2/s,del_phi1/s,del_phi2/s,del_alpha/s]
        grad = [i if abs(i) > 1e-5 else 0 for i in grad]
        return grad
    def GD(p0 = [0,m.pi/2,0,0,m.pi/4], nit = 20, step = 1/120, show_calc = False):
        cost0 = lnL(p0)
        s = step
        grad_new = grad(p = p0)
        params_new = p0
        values = []
        for i in range(nit):
            grad_mod = (sum([abs(i)**2 for i in grad_new]))**(.5)
            params_new = [params_new[i]-s*(grad_new[i]/grad_mod) for i in range(len(p0))]
            grad_new = grad(params_new)
            value = [lnL(params_new), params_new, grad_new, i]
            values.append(value)
            if i > (3/10)*nit//1 and i > 10:
                s = 5*step
                if i > (8/10)*nit//1:
                    s = 15*step
            if show_calc == True:
                print(value,'\n ')
        return params_new

    # GDresult = GDlib(lnL, params0, bounds = bounds)
    opt = GDlib(lnL, params0, bounds = bounds)
    opt_params = opt.x
    
    # now to find the Fidelity
    opt_states_raw = Creating_states(opt_params)[0]              #new states
    
    fid0_sym, fid1_sym = (abs(opt_states_raw[0].H*states[0])**2).evalf(), (abs(opt_states_raw[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid0_cross_sym, fid1_cross_sym = (abs(opt_states_raw[1].H*states[0])**2).evalf(), (abs(opt_states_raw[0].H*states[1])**2).evalf()
    fid0_cross, fid1_cross = complex(fid0_cross_sym[0]).real, complex(fid1_cross_sym[0]).real

    flip_algo = 0
    if fid0_cross + fid1_cross > fid0+fid1:                     # adding zero because of type being matrix..
        opt_params_1 = [opt_params[1],opt_params[0],opt_params[2],opt_params[3],opt_params[4]]        # reversing oder of theta1 and theta2 if cross fidelities are higher.
        flip_algo= 1
        # delta = abs(opt_params_1[0]-opt_params_1[1])/4                                                  # creating a buffer for new bounds
        # bound2 = [(i-delta, i+delta) for i in opt_params_1]                                             # buffer of size delta around each parameter
        # opt_2 = GDlib(lnL, opt_params_1, bounds = bound2)
        # opt_params_2 = opt_2.x
        # opt_params= opt_params_2
        opt_params_2 = GD(opt_params_1)
        opt_params= opt_params_2

    opt_states = Creating_states(opt_params)[0]         #new correctly switched states
    fid0_sym, fid1_sym = (abs(opt_states[0].H*states[0])**2).evalf(), (abs(opt_states[1].H*states[1])**2).evalf()
    fid0, fid1 = complex(fid0_sym[0]).real, complex(fid1_sym[0]).real
    fid = [fid0, fid1]

    if show_calcs == True:
        # pprint(rho)
        # print("collapse vector",nj_vec, "\ngiven parameters\n", params, "\noptimal parameters\n", opt_params)
        # print('\noriginal states: \n' )
        # pprint(states)
        print('\n raw states:' )
        print(list(opt_states_raw[0]))
        print(list(opt_states_raw[1]))
        print('\n final states:')
        print(list(opt_states[0]))
        print(list(opt_states[1]))
        print("\n raw params ", opt.x)
        if flip_algo == 1:
            print("\n intermediate params ", opt_params_1)
        print("\n final params ", opt_params)
    
        if flip_algo == 1:
            print("\n Fidelities raw", [fid0_cross, fid1_cross])
        print("\n Fidelities final",fid)
        
    return [opt_params,opt_states,fid]


In [28]:
#Rough
pr0 = fn.pr0
nj_vec = fn.num_experiment()[-1]
# print(nj_vec)
lnL_list = [nj_vec[i]*sym.log(pr0[i]) for i in range(len(pr0))]     
lnL_abst = sum(lnL_list).evalf()                                  
def lnL(sp = [0,m.pi/2, 0, 0 , m.pi/4]):                                    #s_p stands for search parameters
    lnL = lnL_abst.subs({theta1: sp[0], theta2: sp[1], alpha: sp[2]})
    neg_likelihood = -(lnL.evalf())
    return neg_likelihood 

In [35]:
theta1, theta2, phi1, phi2, alpha = sym.symbols('theta1 theta2 phi1 phi2 alpha')
c1, c2, s1, s2, w1, w2 = sym.cos(theta1), sym.cos(theta2), sym.sin(theta1), sym.sin(theta2), sym.cos(alpha)**2, sym.sin(alpha)**2

# print(pr0)
# print([m.log((i.subs({theta1: 0, theta2: m.pi/2, alpha: m.pi/4}))) for i in pr0])

# print(nj_vec)
# print(lnL([0,m.pi/2,m.pi/4]))
# print(lnL([0,m.pi/2, 0, 0 , m.pi/4]))

res3 = GDlib(lnL, [0,.5,.1])
# res5 = GDlib(lnL, [0,.5,0, m.pi/9, .1])
print(res3, '\n')
# print(res5, '\n')

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''